In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = ( 
 SparkSession
 .builder
    .master("local[*]")
 .appName('spark_sql')
 .getOrCreate()
)

df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-08.csv')
)

In [ ]:
df.printSchema()

In [ ]:
df.createOrReplaceTempView('combustiveis')

In [ ]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` IS NOT NULL
""").show()

In [ ]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`,
        `Unidade de Medida`
    from combustiveis
""")

In [ ]:
view_precos.createOrReplaceTempView('view_precos')
view_precos.show(5)

In [ ]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
   from view_precos
   group by all
   order by diferenca DESC
""")
df_diferenca_precos.show(5)